# 지니차트 200 크롤링 데이터 가져오기

1. 인터넷에서 데이터 가져오기

In [1]:
import requests
import pandas as pd

In [2]:
# Genie top200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [5]:
# 사이트 크롤링 막아놔
# 유저 어전트 이용 헤더 설정. 나는 브라우징을 해서 데이터를 수집중

# Chrome User-Agent
header = {'User-Agent':
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
}
req = requests.get(url, headers=header)
html = req.text
# html

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

찾으려고 하는 데이터의 노드 찾기

In [7]:
trs = soup.find_all('tr')
len(trs)

51

In [8]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [9]:
# 개발자 도구에서 찾아주는 Selector
tbody = soup.select_one('#body-content > div.newest-list > div > table > tbody')
trs = tbody.select('tr')
len(trs)

50

* XPath 값은 아래와 같으나 BeautifulSoup은 지원하지 않음  
//*[@id="body-content"]/div[6]/div/table/tbody


3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출
    - rank, title, artist, album

In [10]:
tr = trs[0]

In [11]:
# rank
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [12]:
rank = tr.select_one('.number').get_text().split('\n')[0]
rank

'1'

In [13]:
# title
tr.select_one('.title.ellipsis').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                Counting Stars (Feat. Beenzino)'

In [14]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'Counting Stars (Feat. Beenzino)'

In [15]:
# artist
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

"BE'O (비오)"

In [16]:
# album
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'Counting Stars'

4. 한 페이지에 있는 모든 데이터를 반복문으로 가져와 데이터프레임 만들기

In [20]:
# 4. 반복문을 통해 데이터 프레임 만들기
# 데이터 프레임 만들기 
# 걍 2차원으로 만들고 때려 넣어
# 줄 단위로 가져오고 줄 단위로 리스트로 어펜드
# 리스트 안에 리스트가 들어가 . 

rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    rank_list.append(rank) 
    title_list.append(title) 
    artist_list.append(artist) 
    album_list.append(album)

In [18]:
df = pd.DataFrame({
    '순위':rank_list, '제목':title_list, '가수':artist_list, '앨범':album_list
})
df.head()

,순위,제목,가수,앨범
0,1,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,ELEVEN,IVE (아이브),ELEVEN


In [22]:
# 2번방법

lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank, title, artist, album])

len(lines), len(lines[0])

(50, 4)

In [25]:
df = pd.DataFrame(lines, columns=['순위', '제목', '가수', '앨범'])
df.head()

,순위,제목,가수,앨범
0,1,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,ELEVEN,IVE (아이브),ELEVEN


3. 딕셔너리의 리스트로 만들어. 제이슨으로 나오는 걸로 보이는데.   
딕 타입으로 나오면 컬럼명 지정 없이 똑같이 만들수 있어  
데이터 프레임을 배우면, 크롤링을 배웠을때 약 3가지 정도의 방법

1. 각각의 반복문 및 어펜드는 귀찮아 공리스트 방법
2. 두번째 방법도 편한데 . 리스트안에 리스트면 2D 데이터 프레임 가능
3. 딕셔너리 타입으로 만들면,,,(아무래도 제이슨 맞나 [ {키:밸류}]) 딱봐도 제이슨 맞는데) 연구할것

In [26]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    d = {'순위':rank, '제목':title, '가수':artist, '앨범':album}
    lines.append(d)

lines[:3]

[{'순위': '1',
  '제목': 'Counting Stars (Feat. Beenzino)',
  '가수': "BE'O (비오)",
  '앨범': 'Counting Stars'},
 {'순위': '2',
  '제목': '회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom)',
  '가수': 'sokodomo',
  '앨범': '쇼미더머니 10 Episode 2'},
 {'순위': '3',
  '제목': '리무진 (Feat. MINO) (Prod. by GRAY)',
  '가수': "BE'O (비오)",
  '앨범': '쇼미더머니 10 Episode 3'}]

In [27]:
df = pd.DataFrame(lines)
df.head()

,순위,제목,가수,앨범
0,1,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,ELEVEN,IVE (아이브),ELEVEN


5. 모든 페이지의 데이터 가져오기

In [29]:
lines = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')

    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        lines.append([rank, title, artist, album])

In [30]:
df = pd.DataFrame(lines, columns=['순위', '제목', '가수', '앨범'])
df.tail()

,순위,제목,가수,앨범
195,196,18세 순이,이찬원,내일은 미스터트롯 결승전 베스트
196,197,별거 없던 그 하루로,임창정,별거 없던 그 하루로
197,198,Shape Of You,Ed Sheeran,÷ (Deluxe)
198,199,밤하늘의 저 별처럼,헤이즈 (Heize) & 펀치 (Punch),밤하늘의 저 별처럼 (브람스를 좋아하세요? OST 스페셜 트랙) (SBS 월화드라마)
199,200,어디에도,엠씨더맥스 (M.C the MAX),pathos


In [31]:
df.to_csv('지니챠트_Top200(211223).csv', index=False)

In [ ]:
# 선생님이 좋아하는 타입의 데이터 프레임 만들기 

# 도로명 주소 이야기  위도, 경도 정보 

# place 도로명 주소  : 행안부 도로명주소 api 이요 얻기
# 위도 경도 정보는 카카오 rest api 이용하면 얻을 수 있어.   
# 폴리움 실습처럼 위경도 자료가 [위도, 경도] 로 나올 수 있는데. 그럴때 2번 개별 단위로 리스트 뽑기 가 좋을 수 있어. 

# df.to_csv이용 ('지니차트_Top200(211223).csv', index=False)


In [ ]:
# 항상 CRUD알고 있어야 해